In [1]:
from config import *
from dataset import ClassificationDataset

import functools

internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "rfcls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model()
Valid:         AUC: 0.7987 | AUPR: 0.2568 | Acc: 88.05%
best model()
Valid:         AUC: 0.7987 | AUPR: 0.2568 | Acc: 88.05%
Internal test: AUC: 0.8242 | AUPR: 0.3014 | Acc: 88.56%
External test: AUC: 0.7922 | AUPR: 0.2507 | Acc: 87.20%


In [3]:
from sklearn.naive_bayes import ComplementNB

model_fn = ComplementNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model()
Valid:         AUC: 0.7771 | AUPR: 0.2067 | Acc: 84.08%
best model()
Valid:         AUC: 0.7771 | AUPR: 0.2067 | Acc: 84.08%
Internal test: AUC: 0.7796 | AUPR: 0.2228 | Acc: 84.11%
External test: AUC: 0.7609 | AUPR: 0.1957 | Acc: 82.77%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto',
    n_neighbors=1, )
param_grid_list = [{
    'p': [1, 2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(p=1)
Valid:         AUC: 0.7168 | AUPR: 0.2896 | Acc: 93.45%
model(p=2)
Valid:         AUC: 0.7080 | AUPR: 0.2688 | Acc: 93.08%
best model(p=1)
Valid:         AUC: 0.7168 | AUPR: 0.2896 | Acc: 93.45%
Internal test: AUC: 0.7042 | AUPR: 0.2713 | Acc: 92.58%
External test: AUC: 0.7155 | AUPR: 0.2978 | Acc: 93.38%


In [5]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)

model(C=1.0)
Valid:         AUC: 0.7265 | AUPR: 0.4074 | Acc: 95.38%
model(C=10.0)
Valid:         AUC: 0.7283 | AUPR: 0.4004 | Acc: 95.29%
model(C=100.0)
Valid:         AUC: 0.7299 | AUPR: 0.4014 | Acc: 95.29%
best model(C=100.0)
Valid:         AUC: 0.7299 | AUPR: 0.4014 | Acc: 95.29%
Internal test: AUC: 0.7321 | AUPR: 0.4390 | Acc: 95.30%
External test: AUC: 0.7158 | AUPR: 0.3820 | Acc: 94.92%


In [6]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, 
    class_weight='balanced', kernel='linear',
    )
param_grid_list = [{
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(C=0.01)
Valid:         AUC: 0.8223 | AUPR: 0.2478 | Acc: 85.20%
model(C=0.1)
Valid:         AUC: 0.8290 | AUPR: 0.2571 | Acc: 85.70%
model(C=1)
Valid:         AUC: 0.8298 | AUPR: 0.2587 | Acc: 85.84%
best model(C=1)
Valid:         AUC: 0.8298 | AUPR: 0.2587 | Acc: 85.84%
Internal test: AUC: 0.8374 | AUPR: 0.2839 | Acc: 86.16%
External test: AUC: 0.8362 | AUPR: 0.2686 | Acc: 85.47%


In [7]:
# from sklearn.neural_network import MLPClassifier

# model_fn = functools.partial(MLPClassifier, random_state=SEED, max_iter=1000)
# param_grid_list = [{
#     'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
#     'activation': ['logistic', 'relu'],
#     'solver': ['adam', 'sgd'],
#     'alpha': [0.0001, 0.001, 0.01],
# }]
# best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
# metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

In [8]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(criterion=gini)
Valid:         AUC: 0.7975 | AUPR: 0.2325 | Acc: 85.24%
model(criterion=log_loss)
Valid:         AUC: 0.8032 | AUPR: 0.2300 | Acc: 84.72%
model(criterion=entropy)
Valid:         AUC: 0.8032 | AUPR: 0.2300 | Acc: 84.72%
best model(criterion=log_loss)
Valid:         AUC: 0.8032 | AUPR: 0.2300 | Acc: 84.72%
Internal test: AUC: 0.8128 | AUPR: 0.2485 | Acc: 84.31%
External test: AUC: 0.7960 | AUPR: 0.2279 | Acc: 84.13%


In [9]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.8249 | AUPR: 0.2970 | Acc: 89.41%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.8252 | AUPR: 0.3003 | Acc: 89.61%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.8270 | AUPR: 0.3024 | Acc: 89.63%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.8249 | AUPR: 0.2931 | Acc: 89.14%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.8271 | AUPR: 0.2999 | Acc: 89.49%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.8298 | AUPR: 0.3047 | Acc: 89.63%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.8249 | AUPR: 0.2931 | Acc: 89.14%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.8271 | AUPR: 0.2999 | Acc: 89.49%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.8298 | AUPR: 0.3047 | Acc: 89.63%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.8298 | AUPR: 0.3047 | Acc: 89.63%
Internal test

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(criterion=friedman_mse)
Valid:         AUC: 0.7337 | AUPR: 0.3911 | Acc: 95.13%
model(criterion=squared_error)
Valid:         AUC: 0.7336 | AUPR: 0.3906 | Acc: 95.12%
best model(criterion=friedman_mse)
Valid:         AUC: 0.7337 | AUPR: 0.3911 | Acc: 95.13%
Internal test: AUC: 0.7466 | AUPR: 0.4436 | Acc: 95.28%
External test: AUC: 0.7372 | AUPR: 0.4107 | Acc: 95.13%


In [11]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.7352 | AUPR: 0.4009 | Acc: 95.24%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.7359 | AUPR: 0.3999 | Acc: 95.22%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.7344 | AUPR: 0.4024 | Acc: 95.27%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.7359 | AUPR: 0.3999 | Acc: 95.22%
Internal test: AUC: 0.7418 | AUPR: 0.4351 | Acc: 95.21%
External test: AUC: 0.7337 | AUPR: 0.3962 | Acc: 94.96%
